In [1]:
import pandas as pd
import pandas_ta as ta

In [39]:
df = pd.read_csv("EURUSD60.csv")

In [12]:
df.iloc[35000]

date      2003.07.11
time           15:00
open          1.1309
high          1.1318
low           1.1284
close          1.129
volume           600
Name: 35000, dtype: object

In [14]:
df.iloc[161036]

date      2023.11.03
time           23:00
open         1.07242
high         1.07321
low          1.07238
close        1.07306
volume          2116
Name: 161036, dtype: object

In [3]:
df.head()

,date,time,open,high,low,close,volume
0,1971.01.04,02:00,0.5369,0.5369,0.5369,0.5369,1
1,1971.01.05,02:00,0.5366,0.5366,0.5366,0.5366,1
2,1971.01.06,02:00,0.5365,0.5365,0.5365,0.5365,1
3,1971.01.07,02:00,0.5368,0.5368,0.5368,0.5368,1
4,1971.01.08,02:00,0.5371,0.5371,0.5371,0.5371,1


In [4]:
df.describe()

,open,high,low,close,volume
count,161037.000000,161037.000000,161037.000000,161037.000000,161037.000000
mean,1.179324,1.180410,1.178264,1.179328,2070.930240
std,0.166850,0.166963,0.166736,0.166847,2442.960577
min,0.536500,0.536500,0.536500,0.536500,1.000000
25%,1.081880,1.082900,1.080920,1.081900,407.000000
50%,1.177270,1.178100,1.176460,1.177270,1104.000000
75%,1.303400,1.304500,1.302250,1.303400,2939.000000
max,1.601500,1.603800,1.599200,1.601600,37898.000000


In [40]:
df['RSI'] = ta.rsi(df.close, length=12)
df['EMA'] = ta.ema(df.close, length=150)
df = df[35000:161036]

In [41]:
df = df.reset_index()

In [26]:
df.head()

,level_0,index,date,time,open,high,low,close,volume,RSI,EMA
0,35000,105000,2014.10.28,20:00,1.27435,1.27494,1.27375,1.27382,2820,63.472255,1.270082
1,35001,105001,2014.10.28,21:00,1.27383,1.27472,1.27348,1.27359,1875,61.651231,1.270128
2,35002,105002,2014.10.28,22:00,1.27359,1.27384,1.27322,1.27383,1599,62.864056,1.270177
3,35003,105003,2014.10.28,23:00,1.27385,1.27387,1.27315,1.27336,1315,58.885459,1.270219
4,35004,105004,2014.10.29,00:00,1.27336,1.27360,1.27309,1.27327,498,58.117099,1.270260


In [42]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [28]:
df.head()

,level_0,index,date,time,open,high,low,close,volume,RSI,EMA,EMASignal
0,35000,105000,2014.10.28,20:00,1.27435,1.27494,1.27375,1.27382,2820,63.472255,1.270082,0
1,35001,105001,2014.10.28,21:00,1.27383,1.27472,1.27348,1.27359,1875,61.651231,1.270128,0
2,35002,105002,2014.10.28,22:00,1.27359,1.27384,1.27322,1.27383,1599,62.864056,1.270177,0
3,35003,105003,2014.10.28,23:00,1.27385,1.27387,1.27315,1.27336,1315,58.885459,1.270219,0
4,35004,105004,2014.10.29,00:00,1.27336,1.27360,1.27309,1.27327,498,58.117099,1.270260,0


In [43]:
def generate_signal(df,l,backcandles, gap, zone_threshold, price_diff_threshold):
    max_price = df.high[l-backcandles:l-gap].max()
    min_price = df.low[l-backcandles:l-gap].min()
    index_max = df.high[l-backcandles:l-gap].idxmax()
    index_min = df.low[l-backcandles:l-gap].idxmin()
    price_diff = max_price - min_price

    if (df.EMASignal[l] == 2 
        and (index_min < index_max) 
        and price_diff>price_diff_threshold): 
        l1 = max_price - 0.62 * price_diff # position entry 0.62
        l2 = max_price - 0.78 * price_diff # SL 0.78
        l3 = max_price - 0. * price_diff # TP
        if abs(df.close[l]-l1) < zone_threshold and df.high[l-gap:l].min()>l1:
            return (2, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)
        
    elif (df.EMASignal[l] == 1 
          and (index_min > index_max) 
          and price_diff>price_diff_threshold): 
        l1 = min_price + 0.62 * price_diff # position entry 0.62
        l2 = min_price + 0.78 * price_diff # SL 0.78
        l3 = min_price + 0. * price_diff # TP
        if abs(df.close[l]-l1) < zone_threshold and df.low[l-gap:l].max()<l1:
            return (1, l2, l3, index_min, index_max)
        else:
            return (0,0,0,0,0)
    
    else:
        return (0,0,0,0,0)


gap_candles = 5
backcandles = 25
signal = [0 for i in range(len(df))]
TP = [0 for i in range(len(df))]
SL = [0 for i in range(len(df))]
MinSwing = [0 for i in range(len(df))]
MaxSwing = [0 for i in range(len(df))]

for row in range(backcandles, len(df)):
    gen_sig = generate_signal(df, row, backcandles=backcandles, gap=gap_candles, zone_threshold=0.001, price_diff_threshold=0.01)
    signal[row] = gen_sig[0]
    SL[row] = gen_sig[1]
    TP[row] = gen_sig[2]
    MinSwing[row] = gen_sig[3]
    MaxSwing[row] = gen_sig[4]
    
df['signal'] = signal
df['SL'] = SL
df['TP'] = TP
df['MinSwing'] = MinSwing
df['MaxSwing'] = MaxSwing

In [44]:
df[df.signal !=0][:10]

,index,date,time,open,high,low,close,volume,RSI,EMA,EMASignal,signal,SL,TP,MinSwing,MaxSwing
216,35216,2003.07.24,15:00,1.1477,1.1477,1.1434,1.1442,568,41.276879,1.135010,2,2,1.141532,1.1498,191,210
251,35251,2003.07.28,02:00,1.1500,1.1505,1.1487,1.1491,283,47.686617,1.140081,2,2,1.146798,1.1553,240,242
252,35252,2003.07.28,03:00,1.1494,1.1498,1.1482,1.1483,328,44.297872,1.140189,2,2,1.146798,1.1553,240,242
253,35253,2003.07.28,04:00,1.1483,1.1490,1.1464,1.1477,428,41.863814,1.140289,2,2,1.146798,1.1553,240,242
254,35254,2003.07.28,05:00,1.1474,1.1480,1.1465,1.1476,360,41.449712,1.140386,2,2,1.146798,1.1553,240,242
435,35435,2003.08.06,18:00,1.1376,1.1386,1.1353,1.1360,495,40.321288,1.135234,2,2,1.133406,1.1430,410,427
469,35469,2003.08.08,04:00,1.1365,1.1371,1.1355,1.1359,304,44.727560,1.135486,2,2,1.134100,1.1419,450,459
470,35470,2003.08.08,05:00,1.1360,1.1365,1.1352,1.1357,394,43.619990,1.135489,2,2,1.134100,1.1419,450,459
471,35471,2003.08.08,06:00,1.1357,1.1367,1.1353,1.1363,352,47.846565,1.135500,2,2,1.134100,1.1419,450,459
472,35472,2003.08.08,07:00,1.1363,1.1365,1.1354,1.1361,296,46.576866,1.135508,2,2,1.134100,1.1419,450,459


In [45]:
df = df.rename(columns={"open": "Open", "high":"High", "low":"Low", "close": "Close", "volume":"Volume"})
def SIGNAL():
    return df.signal

In [46]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.99 #1000
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
       
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.SL[-1]
            tp1 = self.data.TP[-1]
            tp2 = tp1-(tp1-self.data.Close[-1])/2
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SL[-1]
            tp1 = self.data.TP[-1]
            tp2 = tp1+(self.data.Close[-1]-tp1)/2
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=100, margin=1/100, commission=0.0000)
stat = bt.run()
stat

C:\Users\ryanh\AppData\Local\Temp\ipykernel_8992\4137819960.py:27: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=100, margin=1/100, commission=0.0000)


Start                                     0.0
End                                  126035.0
Duration                             126035.0
Exposure Time [%]                    1.185376
Equity Final [$]                     0.009291
Equity Peak [$]                    239.165574
Return [%]                         -99.990709
Buy & Hold Return [%]               -5.014172
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.996115
Avg. Drawdown [%]                  -32.965117
Max. Drawdown Duration               123854.0
Avg. Drawdown Duration           17931.285714
# Trades                                294.0
Win Rate [%]                        29.591837
Best Trade [%]                       1.319943
Worst Trade [%]                      -0.45932
Avg. Trade [%]                    

In [52]:
bt.plot()

c:\Users\ryanh\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\ryanh\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1307', ...)

In [37]:
stat.get("Return [%]")

-99.99046420000263